In [1]:
import dill
import pandas as pd

# 读取数据

In [2]:
train_data = {
    'trajectories': pd.read_csv('../trajectories(table 5)_training.csv'),
    'volume': pd.read_csv('../volume(table 6)_training.csv'),
    'avg_travel_time': pd.read_csv('../training_20min_avg_travel_time.csv'),
    'avg_volume': pd.read_csv('../training_20min_avg_volume.csv'),
}

test_data = {
    'trajectories': pd.read_csv('../trajectories(table 5)_training.csv'),
    'volume': pd.read_csv('../volume(table 6)_training.csv'),
    'avg_travel_time': pd.read_csv('../test1_20min_avg_travel_time.csv'),
    'avg_volume': pd.read_csv('../test1_20min_avg_volume.csv'),
}

links = pd.read_csv('../links (table 3).csv')
routes = pd.read_csv('../routes (table 4).csv')
weather = pd.concat([
    pd.read_csv('../weather (table 7)_training_update.csv'),
    pd.read_csv('../weather (table 7)_test1.csv')])

train_trajectories = train_data['trajectories']
train_volume = train_data['volume']

train_avg_travel_time = train_data['avg_travel_time']

train_avg_volume = train_data['avg_volume']

test_avg_travel_time = test_data['avg_travel_time']

test_avg_volume = test_data['avg_volume']

In [3]:
# 处理天气数据 插值填充每小时数据

weather.date = pd.to_datetime(weather.date)
weather['time'] = weather.date + pd.to_timedelta(weather.hour, unit='h')

weather = weather.set_index(['time'])
weather = weather.reindex(pd.date_range(start= weather.date.min(), end=weather.date.max(), freq='1H'), fill_value='NaN')

weather['date'] = pd.to_datetime([t.date() for t in weather.index])
weather['year'] = [t.year for t in weather.index]
weather['day_of_year'] = [t.dayofyear for t in weather.index]
weather['hour'] = [t.hour for t in weather.index]
for col in ['pressure', 'sea_pressure', 'wind_direction', 'wind_speed', 'temperature', 'rel_humidity', 'precipitation']:
    weather[col] = weather[col].astype(float).interpolate()
    
weather = weather.drop(['date'], axis=1)

In [4]:
# 拆分时间窗

def split_time_window(time_window):
    time_start = [time[1:-1].split(',')[0] for time in time_window]
    time_end = [time[1:-1].split(',')[1] for time in time_window]

    return pd.to_datetime(pd.Series(time_start)), pd.to_datetime(pd.Series(time_end))

train_avg_volume['time_start'], train_avg_volume['time_end'] = split_time_window(train_avg_volume.time_window)
train_avg_volume = train_avg_volume.drop(['time_window'], axis=1)

train_avg_travel_time['time_start'], train_avg_travel_time['time_end'] = split_time_window(train_avg_travel_time.time_window)
train_avg_travel_time = train_avg_travel_time.drop(['time_window'], axis=1)

test_avg_volume['time_start'], test_avg_volume['time_end'] = split_time_window(test_avg_volume.time_window)
test_avg_volume = test_avg_volume.drop(['time_window'], axis=1)

test_avg_travel_time['time_start'], test_avg_travel_time['time_end'] = split_time_window(test_avg_travel_time.time_window)
test_avg_travel_time = test_avg_travel_time.drop(['time_window'], axis=1)

In [5]:
dill.dump_session("data.pickle")

In [6]:
'''train_path = pd.read_csv('../train_avg_travel_time_4h&path.csv')
train_path = train_path.drop(['Unnamed: 0','time_window_start',
                              'time_start_of_day',
                              'Traffic_jam_probability_20min1', 
                              'Traffic_jam_probability_20min2',
       'Traffic_jam_probability_20min3', 'Traffic_jam_probability_20min4',
       'Traffic_jam_probability_20min5', 'Traffic_jam_probability_20min6',
       'Traffic_jam_probability_20min7', 'Traffic_jam_probability_20min8',
       'Traffic_jam_probability_20min9', 'Traffic_jam_probability_20min10',
       'Traffic_jam_probability_20min11', 'Traffic_jam_probability_20min12',
       'Traffic_jam_probability_40min1', 'Traffic_jam_probability_40min2',
       'Traffic_jam_probability_40min3', 'Traffic_jam_probability_40min4',
       'Traffic_jam_probability_40min5', 'Traffic_jam_probability_40min6',
       'Traffic_jam_probability_1h1', 'Traffic_jam_probability_1h2',
       'Traffic_jam_probability_1h3', 'Traffic_jam_probability_1h4',
       'Traffic_jam_probability_2h1', 'Traffic_jam_probability_2h2',
                              'path_time_median_20min1',
       'path_time_median_20min2', 'path_time_median_20min3',
       'path_time_median_20min4', 'path_time_median_20min5',
       'path_time_median_20min6', 'path_time_median_20min7',
       'path_time_median_20min8', 'path_time_median_20min9',
       'path_time_median_20min10', 'path_time_median_20min11',
       'path_time_median_20min12', 'path_time_median_40min1',
       'path_time_median_40min2', 'path_time_median_40min3',
       'path_time_median_40min4', 'path_time_median_40min5',
       'path_time_median_40min6', 'path_time_median_1h1',
       'path_time_median_1h2', 'path_time_median_1h3', 'path_time_median_1h4',
       'path_time_median_2h1', 'path_time_median_2h2'], axis = 1)

test_path = pd.read_csv('../test_avg_travel_time_4h&path.csv')
test_path = test_path.drop(['Unnamed: 0','time_window_start','time_start_of_day',
                            'Traffic_jam_probability_20min1', 'Traffic_jam_probability_20min2',
       'Traffic_jam_probability_20min3', 'Traffic_jam_probability_20min4',
       'Traffic_jam_probability_20min5', 'Traffic_jam_probability_20min6',
       'Traffic_jam_probability_20min7', 'Traffic_jam_probability_20min8',
       'Traffic_jam_probability_20min9', 'Traffic_jam_probability_20min10',
       'Traffic_jam_probability_20min11', 'Traffic_jam_probability_20min12',
       'Traffic_jam_probability_40min1', 'Traffic_jam_probability_40min2',
       'Traffic_jam_probability_40min3', 'Traffic_jam_probability_40min4',
       'Traffic_jam_probability_40min5', 'Traffic_jam_probability_40min6',
       'Traffic_jam_probability_1h1', 'Traffic_jam_probability_1h2',
       'Traffic_jam_probability_1h3', 'Traffic_jam_probability_1h4',
       'Traffic_jam_probability_2h1', 'Traffic_jam_probability_2h2',
                            'path_time_median_20min1',
       'path_time_median_20min2', 'path_time_median_20min3',
       'path_time_median_20min4', 'path_time_median_20min5',
       'path_time_median_20min6', 'path_time_median_20min7',
       'path_time_median_20min8', 'path_time_median_20min9',
       'path_time_median_20min10', 'path_time_median_20min11',
       'path_time_median_20min12', 'path_time_median_40min1',
       'path_time_median_40min2', 'path_time_median_40min3',
       'path_time_median_40min4', 'path_time_median_40min5',
       'path_time_median_40min6', 'path_time_median_1h1',
       'path_time_median_1h2', 'path_time_median_1h3', 'path_time_median_1h4',
       'path_time_median_2h1', 'path_time_median_2h2'], axis = 1)'''

"train_path = pd.read_csv('../train_avg_travel_time_4h&path.csv')\ntrain_path = train_path.drop(['Unnamed: 0','time_window_start',\n                              'time_start_of_day',\n                              'Traffic_jam_probability_20min1', \n                              'Traffic_jam_probability_20min2',\n       'Traffic_jam_probability_20min3', 'Traffic_jam_probability_20min4',\n       'Traffic_jam_probability_20min5', 'Traffic_jam_probability_20min6',\n       'Traffic_jam_probability_20min7', 'Traffic_jam_probability_20min8',\n       'Traffic_jam_probability_20min9', 'Traffic_jam_probability_20min10',\n       'Traffic_jam_probability_20min11', 'Traffic_jam_probability_20min12',\n       'Traffic_jam_probability_40min1', 'Traffic_jam_probability_40min2',\n       'Traffic_jam_probability_40min3', 'Traffic_jam_probability_40min4',\n       'Traffic_jam_probability_40min5', 'Traffic_jam_probability_40min6',\n       'Traffic_jam_probability_1h1', 'Traffic_jam_probability_1h2',\n    